In [19]:
#from keys import ml_prod_servicekey as ml_key
from keys import ml_servicekey_wrongobject, ml_servicekey_hole, ml_servicekey_scratch
from ml_interface import MLFoundationClient, DefectCode
from database_interface import DBClient
from image_interface import ImageDBClient

In [18]:
import os
import time

input_dir = "input/"
sleep_time = 1 # seconds
approved_extensions = [".jpg", ".jpeg"]

files = []
while 1:
    curr_files = os.listdir(input_dir)
    new_files = [x for x in curr_files if x not in files and 
                 any([x.endswith(ext) for ext in approved_extensions])]
    if len(new_files) >= 2:
        files = curr_files
    print (new_files)
    time.sleep(sleep_time)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['test-photo.jpg']
['test-photo.jpg']
['test-photo.jpg']
['test-photo.jpg']
['test-photo.jpg']
['test-photo.jpg']
['test-photo.jpg', 'test-photo2.jpeg']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 

In [34]:
from PIL import Image
i1 = Image.open("test-photo3.jpg")
i1.size

(3024, 4032)

In [33]:
i1.thumbnail((1440, 1920), Image.ANTIALIAS)
i1.save("test-photo3-resized.jpg", "JPEG")

In [2]:
# init the clients
db_client = DBClient()
#ml_client = MLFoundationClient(ml_key, offline = True)
im_client = ImageDBClient()

In [3]:
img_path = "test-photo2.jpeg"
img_path2 = "test-photo.jpg"
img = open(img_path, "rb")
img2 = open(img_path2, "rb")

In [6]:
# use this key to upload to the database
key = im_client.upload(img_path)
key2 = im_client.upload(img_path2)

In [7]:
# for every model
clientWrongObject = MLFoundationClient(ml_servicekey_wrongobject)
defectWrongObject = clientWrongObject.modelPredictOne(img_path, "wrong_object","WrongObjectModel", DefectCode.WrongProduct)
defectWrongObject2 = clientWrongObject.modelPredictOne(img_path2, "wrong_object","WrongObjectModel", DefectCode.WrongProduct)

clientHole = MLFoundationClient(ml_servicekey_hole)
defectHole = clientHole.modelPredictOne(img_path, "hole","HoleModel", DefectCode.Hole)
defectHole2 = clientHole.modelPredictOne(img_path2, "hole","HoleModel", DefectCode.Hole)

clientScratch = MLFoundationClient(ml_servicekey_scratch)
defectScratch = clientScratch.modelPredictOne(img_path, "scratch","ScratchModel", DefectCode.Scratch)
defectScratch2 = clientScratch.modelPredictOne(img_path2, "scratch","ScratchModel", DefectCode.Scratch)

defects = list(set(defectHole+defectHole2+defectScratch+defectScratch2+defectWrongObject+defectWrongObject2))

#unite the lists


#make the json
imageInfo= {
    'image1' : key,
    'image2' : key2,
    'defects' : defects     
}
print(imageInfo)

{'id': '20bec5fc-fd45-4376-5276-929719db8135', 'predictions': [{'name': 'test-photo2.jpeg', 'results': [{'label': 'right_object', 'score': 0.935053288936615}, {'label': 'wrong_object', 'score': 0.06494668871164322}]}], 'processedTime': '2019-07-03T16:42:36.095727+00:00', 'status': 'DONE'}
{'id': 'f19ffedf-31cc-47fd-7863-df64cab4922a', 'predictions': [{'name': 'test-photo.jpg', 'results': [{'label': 'right_object', 'score': 0.9922181367874146}, {'label': 'wrong_object', 'score': 0.0077818320132792}]}], 'processedTime': '2019-07-03T16:42:38.899700+00:00', 'status': 'DONE'}
{'id': '3eaba295-91ca-45f6-51d7-746dbeeb7308', 'predictions': [{'name': 'test-photo2.jpeg', 'results': [{'label': 'no_hole', 'score': 0.6651802659034729}, {'label': 'hole', 'score': 0.3348197638988495}]}], 'processedTime': '2019-07-03T16:42:42.297105+00:00', 'status': 'DONE'}
{'id': 'b1d51b78-d753-4701-5bac-dca18bce523e', 'predictions': [{'name': 'test-photo.jpg', 'results': [{'label': 'no_hole', 'score': 0.98934990167

In [8]:
db_client.saveMachineLearningResult(imageInfo, productExists=False)

New register inserted succesfully


True